第一波特征：lr 0.71。categorical feature 做target、onehot、label encoding，加上normal features，60维

第二波特征：lr 0.75。对连续型，数值型变量做离散化处理。998维

第三波特征：lr 0.76。引入bureau特征。1468维

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import category_encoders as ce
from sklearn.preprocessing import LabelBinarizer, StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2, SelectFromModel, mutual_info_classif
from sklearn.ensemble import ExtraTreesClassifier

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
train = pd.read_csv('./data/rawdata/application_train.csv.zip',compression='zip')
test = pd.read_csv('./data/rawdata/application_test.csv.zip',compression='zip')

In [3]:
train.shape
train.head()
train.tail()
test.shape
test.tail()

(307511, 122)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
307506,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307507,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307508,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,0,0,0,0,1.0,0.0,0.0,1.0,0.0,1.0
307509,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
307510,456255,0,Cash loans,F,N,N,0,157500.0,675000.0,49117.5,...,0,0,0,0,0.0,0.0,0.0,2.0,0.0,1.0


(48744, 121)

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
48739,456221,Cash loans,F,N,Y,0,121500.0,412560.0,17473.5,270000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
48740,456222,Cash loans,F,N,N,2,157500.0,622413.0,31909.5,495000.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
48741,456223,Cash loans,F,Y,Y,1,202500.0,315000.0,33205.5,315000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,3.0,1.0
48742,456224,Cash loans,M,N,N,0,225000.0,450000.0,25128.0,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0
48743,456250,Cash loans,F,Y,N,0,135000.0,312768.0,24709.5,270000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0


In [4]:
#  add features from other source
bureau_features = pd.read_csv('./data/rawdata/bureau_feature.csv')
bureau_features.columns = ['bureau_' + c for c in bureau_features.columns]
train = train.merge(bureau_features,how='left',on='SK_ID_CURR')
test = test.merge(bureau_features,how='left',on='SK_ID_CURR')


In [5]:
bureau_features.shape
train.shape
train.head()
train.tail()
test.shape
test.tail()

(305811, 31)

(307511, 152)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CREDIT_CURRENCY_cnt,CREDIT_ACTIVE_cnt,CREDIT_TYPE_cnt,STATUS_set_cnt,latest_STATUS_cnt,CREDIT_CURRENCY_set,CREDIT_ACTIVE_set,CREDIT_TYPE_set,latest_STATUS_set,STATUS_set
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,1.0,2.0,2.0,4.0,2.0,{'currency 1'},"{'Closed', 'Active'}","{'Credit card', 'Consumer credit'}","{'C', '0'}",{'STATUS_set'}
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,1.0,2.0,2.0,0.0,0.0,{'currency 1'},"{'Closed', 'Active'}","{'Credit card', 'Consumer credit'}",{nan},{'STATUS_set'}
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,1.0,1.0,1.0,0.0,0.0,{'currency 1'},{'Closed'},{'Consumer credit'},{nan},{'STATUS_set'}
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,1.0,1.0,1.0,0.0,0.0,{'currency 1'},{'Closed'},{'Consumer credit'},{nan},{'STATUS_set'}


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CREDIT_CURRENCY_cnt,CREDIT_ACTIVE_cnt,CREDIT_TYPE_cnt,STATUS_set_cnt,latest_STATUS_cnt,CREDIT_CURRENCY_set,CREDIT_ACTIVE_set,CREDIT_TYPE_set,latest_STATUS_set,STATUS_set
307506,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307507,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307508,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,1.0,2.0,2.0,2.0,2.0,{'currency 1'},"{'Closed', 'Active'}","{'Credit card', 'Consumer credit'}","{'C', 'X'}",{'STATUS_set'}
307509,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,1.0,1.0,1.0,1.0,1.0,{'currency 1'},{'Closed'},{'Consumer credit'},{'C'},{'STATUS_set'}
307510,456255,0,Cash loans,F,N,N,0,157500.0,675000.0,49117.5,...,1.0,2.0,2.0,1.0,1.0,{'currency 1'},"{'Closed', 'Active'}","{'Credit card', 'Consumer credit'}",{'C'},{'STATUS_set'}


(48744, 151)

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CREDIT_CURRENCY_cnt,CREDIT_ACTIVE_cnt,CREDIT_TYPE_cnt,STATUS_set_cnt,latest_STATUS_cnt,CREDIT_CURRENCY_set,CREDIT_ACTIVE_set,CREDIT_TYPE_set,latest_STATUS_set,STATUS_set
48739,456221,Cash loans,F,N,Y,0,121500.0,412560.0,17473.5,270000.0,...,1.0,2.0,3.0,4.0,3.0,{'currency 1'},"{'Closed', 'Active'}","{'Credit card', 'Microloan', 'Consumer credit'}","{'C', '0', 'X'}",{'STATUS_set'}
48740,456222,Cash loans,F,N,N,2,157500.0,622413.0,31909.5,495000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48741,456223,Cash loans,F,Y,Y,1,202500.0,315000.0,33205.5,315000.0,...,1.0,2.0,3.0,4.0,2.0,{'currency 1'},"{'Closed', 'Active'}","{'Mortgage', 'Credit card', 'Consumer credit'}","{'C', 'X'}",{'STATUS_set'}
48742,456224,Cash loans,M,N,N,0,225000.0,450000.0,25128.0,450000.0,...,1.0,2.0,2.0,6.0,3.0,{'currency 1'},"{'Closed', 'Active'}","{'Credit card', 'Consumer credit'}","{'C', '0', 'X'}",{'STATUS_set'}
48743,456250,Cash loans,F,Y,N,0,135000.0,312768.0,24709.5,270000.0,...,1.0,2.0,2.0,2.0,3.0,{'currency 1'},"{'Closed', 'Active'}","{'Credit card', 'Consumer credit'}","{'C', '0', 'X'}",{'STATUS_set'}


In [6]:
#  add features from other source
cash_features = pd.read_csv('./data/rawdata/cash_features.csv')
cash_features.columns = ['cash_' + c for c in cash_features.columns]

train = train.merge(cash_features,how='left',on='SK_ID_CURR')
test = test.merge(cash_features,how='left',on='SK_ID_CURR')


In [8]:
cash_features.shape
cash_features.columns
train.shape
train.head()
train.tail()
test.shape
test.tail()

(337252, 18)

Index(['SK_ID_CURR', 'MONTHS_BALANCE_min', 'MONTHS_BALANCE_max',
       'MONTHS_BALANCE_cnt', 'CNT_INSTALMENT_min', 'CNT_INSTALMENT_max',
       'CNT_INSTALMENT_set', 'CNT_INSTALMENT_FUTURE_min',
       'CNT_INSTALMENT_FUTURE_max', 'NAME_CONTRACT_STATUS_cntd',
       'NAME_CONTRACT_STATUS_set', 'NAME_CONTRACT_STATUS_latest', 'SK_DPD_max',
       'SK_DPD_cnt0', 'SK_DPD_DEF_max', 'SK_DPD_DEF_cnt0', 'SK_DPD_diff_max',
       'SK_DPD_diff_cnt0'],
      dtype='object')

(307511, 169)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CNT_INSTALMENT_FUTURE_max,NAME_CONTRACT_STATUS_cntd,NAME_CONTRACT_STATUS_set,NAME_CONTRACT_STATUS_latest,SK_DPD_max,SK_DPD_cnt0,SK_DPD_DEF_max,SK_DPD_DEF_cnt0,SK_DPD_diff_max,SK_DPD_diff_cnt0
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,24.0,1.0,Active,NaN,0.0,0.0,0.0,0.0,0.0,0.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,12.0,2.0,"Active,Completed",Completed,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,4.0,2.0,"Active,Completed",Completed,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,48.0,3.0,"Active,Returned to the store,Completed",Completed,0.0,0.0,0.0,0.0,0.0,0.0
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,24.0,3.0,"Active,Signed,Completed","Active,Completed",0.0,0.0,0.0,0.0,0.0,0.0


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CNT_INSTALMENT_FUTURE_max,NAME_CONTRACT_STATUS_cntd,NAME_CONTRACT_STATUS_set,NAME_CONTRACT_STATUS_latest,SK_DPD_max,SK_DPD_cnt0,SK_DPD_DEF_max,SK_DPD_DEF_cnt0,SK_DPD_diff_max,SK_DPD_diff_cnt0
307506,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,8.0,3.0,"Active,Signed,Completed",Completed,0.0,0.0,0.0,0.0,0.0,0.0
307507,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,6.0,2.0,"Active,Completed",NaN,0.0,0.0,0.0,0.0,0.0,0.0
307508,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,6.0,2.0,"Active,Completed",Completed,5.0,1.0,5.0,1.0,0.0,0.0
307509,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,16.0,1.0,Active,NaN,0.0,0.0,0.0,0.0,0.0,0.0
307510,456255,0,Cash loans,F,N,N,0,157500.0,675000.0,49117.5,...,36.0,2.0,"Active,Completed","Active,Completed",5.0,1.0,5.0,1.0,0.0,0.0


(48744, 168)

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CNT_INSTALMENT_FUTURE_max,NAME_CONTRACT_STATUS_cntd,NAME_CONTRACT_STATUS_set,NAME_CONTRACT_STATUS_latest,SK_DPD_max,SK_DPD_cnt0,SK_DPD_DEF_max,SK_DPD_DEF_cnt0,SK_DPD_diff_max,SK_DPD_diff_cnt0
48739,456221,Cash loans,F,N,Y,0,121500.0,412560.0,17473.5,270000.0,...,24.0,2.0,"Active,Completed",Completed,0.0,0.0,0.0,0.0,0.0,0.0
48740,456222,Cash loans,F,N,N,2,157500.0,622413.0,31909.5,495000.0,...,24.0,3.0,"Active,Signed,Completed","Active,Completed",0.0,0.0,0.0,0.0,0.0,0.0
48741,456223,Cash loans,F,Y,Y,1,202500.0,315000.0,33205.5,315000.0,...,12.0,2.0,"Active,Completed","Active,Completed",0.0,0.0,0.0,0.0,0.0,0.0
48742,456224,Cash loans,M,N,N,0,225000.0,450000.0,25128.0,450000.0,...,24.0,3.0,"Active,Signed,Completed",NaN,0.0,0.0,0.0,0.0,0.0,0.0
48743,456250,Cash loans,F,Y,N,0,135000.0,312768.0,24709.5,270000.0,...,12.0,2.0,"Active,Completed","Active,Completed",0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#  add features from other source
credit_features = pd.read_csv('./data/rawdata/credit_features.csv')
credit_features.columns = ['credit_' + c for c in credit_features.columns]

train = train.merge(cash_features,how='left',on='SK_ID_CURR')
test = test.merge(cash_features,how='left',on='SK_ID_CURR')


In [ ]:
credit_features.shape
credit_features.columns
train.shape
train.head()
train.tail()
test.shape
test.tail()

In [ ]:
#  add features from other source
installment_features = pd.read_csv('./data/rawdata/installment_features.csv')
installment_features.columns = ['installment_' + c for c in installment_features.columns]

train = train.merge(cash_features,how='left',on='SK_ID_CURR')
test = test.merge(cash_features,how='left',on='SK_ID_CURR')


In [ ]:
installment_features.shape
installment_features.columns
train.shape
train.head()
train.tail()
test.shape
test.tail()

In [12]:
feature_dict = {'NAME_CONTRACT_TYPE':'category_features', 'CODE_GENDER':'category_features', 'FLAG_OWN_CAR':'boolean_features', 'FLAG_OWN_REALTY':'boolean_features', 'CNT_CHILDREN':'count_features', 'AMT_INCOME_TOTAL':'amount_feautres', 'AMT_CREDIT':'amount_feautres', 'AMT_ANNUITY':'amount_feautres', 'AMT_GOODS_PRICE':'amount_feautres', 'NAME_TYPE_SUITE':'category_features', 'NAME_INCOME_TYPE':'category_features', 'NAME_EDUCATION_TYPE':'category_features', 'NAME_FAMILY_STATUS':'category_features', 'NAME_HOUSING_TYPE':'category_features', 'REGION_POPULATION_RELATIVE':'normal_features', 'DAYS_BIRTH':'amount_feautres', 'DAYS_EMPLOYED':'amount_feautres', 'DAYS_REGISTRATION':'amount_feautres', 'DAYS_ID_PUBLISH':'amount_feautres', 'OWN_CAR_AGE':'amount_feautres', 'FLAG_MOBIL':'boolean_features', 'FLAG_EMP_PHONE':'boolean_features', 'FLAG_WORK_PHONE':'boolean_features', 'FLAG_CONT_MOBILE':'boolean_features', 'FLAG_PHONE':'boolean_features', 'FLAG_EMAIL':'boolean_features', 'OCCUPATION_TYPE':'category_features', 'CNT_FAM_MEMBERS':'count_features', 'REGION_RATING_CLIENT':'category_features', 'REGION_RATING_CLIENT_W_CITY':'category_features', 'WEEKDAY_APPR_PROCESS_START':'category_features', 'HOUR_APPR_PROCESS_START':'category_features', 'REG_REGION_NOT_LIVE_REGION':'boolean_features', 'REG_REGION_NOT_WORK_REGION':'boolean_features', 'LIVE_REGION_NOT_WORK_REGION':'boolean_features', 'REG_CITY_NOT_LIVE_CITY':'boolean_features', 'REG_CITY_NOT_WORK_CITY':'boolean_features', 'LIVE_CITY_NOT_WORK_CITY':'boolean_features', 'ORGANIZATION_TYPE':'category_features', 'EXT_SOURCE_1':'normal_features', 'EXT_SOURCE_2':'normal_features', 'EXT_SOURCE_3':'normal_features', 'APARTMENTS_AVG':'normal_features', 'BASEMENTAREA_AVG':'normal_features', 'YEARS_BEGINEXPLUATATION_AVG':'normal_features', 'YEARS_BUILD_AVG':'normal_features', 'COMMONAREA_AVG':'normal_features', 'ELEVATORS_AVG':'normal_features', 'ENTRANCES_AVG':'normal_features', 'FLOORSMAX_AVG':'normal_features', 'FLOORSMIN_AVG':'normal_features', 'LANDAREA_AVG':'normal_features', 'LIVINGAPARTMENTS_AVG':'normal_features', 'LIVINGAREA_AVG':'normal_features', 'NONLIVINGAPARTMENTS_AVG':'normal_features', 'NONLIVINGAREA_AVG':'normal_features', 'APARTMENTS_MODE':'normal_features', 'BASEMENTAREA_MODE':'normal_features', 'YEARS_BEGINEXPLUATATION_MODE':'normal_features', 'YEARS_BUILD_MODE':'normal_features', 'COMMONAREA_MODE':'normal_features', 'ELEVATORS_MODE':'normal_features', 'ENTRANCES_MODE':'normal_features', 'FLOORSMAX_MODE':'normal_features', 'FLOORSMIN_MODE':'normal_features', 'LANDAREA_MODE':'normal_features', 'LIVINGAPARTMENTS_MODE':'normal_features', 'LIVINGAREA_MODE':'normal_features', 'NONLIVINGAPARTMENTS_MODE':'normal_features', 'NONLIVINGAREA_MODE':'normal_features', 'APARTMENTS_MEDI':'normal_features', 'BASEMENTAREA_MEDI':'normal_features', 'YEARS_BEGINEXPLUATATION_MEDI':'normal_features', 'YEARS_BUILD_MEDI':'normal_features', 'COMMONAREA_MEDI':'normal_features', 'ELEVATORS_MEDI':'normal_features', 'ENTRANCES_MEDI':'normal_features', 'FLOORSMAX_MEDI':'normal_features', 'FLOORSMIN_MEDI':'normal_features', 'LANDAREA_MEDI':'normal_features', 'LIVINGAPARTMENTS_MEDI':'normal_features', 'LIVINGAREA_MEDI':'normal_features', 'NONLIVINGAPARTMENTS_MEDI':'normal_features', 'NONLIVINGAREA_MEDI':'normal_features', 'FONDKAPREMONT_MODE':'category_features', 'HOUSETYPE_MODE':'category_features', 'TOTALAREA_MODE':'normal_features', 'WALLSMATERIAL_MODE':'category_features', 'EMERGENCYSTATE_MODE':'category_features', 'OBS_30_CNT_SOCIAL_CIRCLE':'count_features', 'DEF_30_CNT_SOCIAL_CIRCLE':'count_features', 'OBS_60_CNT_SOCIAL_CIRCLE':'count_features', 'DEF_60_CNT_SOCIAL_CIRCLE':'count_features', 'DAYS_LAST_PHONE_CHANGE':'count_features', 'FLAG_DOCUMENT_2':'boolean_features', 'FLAG_DOCUMENT_3':'boolean_features', 'FLAG_DOCUMENT_4':'boolean_features', 'FLAG_DOCUMENT_5':'boolean_features', 'FLAG_DOCUMENT_6':'boolean_features', 'FLAG_DOCUMENT_7':'boolean_features', 'FLAG_DOCUMENT_8':'boolean_features', 'FLAG_DOCUMENT_9':'boolean_features', 'FLAG_DOCUMENT_10':'boolean_features', 'FLAG_DOCUMENT_11':'boolean_features', 'FLAG_DOCUMENT_12':'boolean_features', 'FLAG_DOCUMENT_13':'boolean_features', 'FLAG_DOCUMENT_14':'boolean_features', 'FLAG_DOCUMENT_15':'boolean_features', 'FLAG_DOCUMENT_16':'boolean_features', 'FLAG_DOCUMENT_17':'boolean_features', 'FLAG_DOCUMENT_18':'boolean_features', 'FLAG_DOCUMENT_19':'boolean_features', 'FLAG_DOCUMENT_20':'boolean_features', 'FLAG_DOCUMENT_21':'boolean_features', 'AMT_REQ_CREDIT_BUREAU_HOUR':'count_features', 'AMT_REQ_CREDIT_BUREAU_DAY':'count_features', 'AMT_REQ_CREDIT_BUREAU_WEEK':'count_features', 'AMT_REQ_CREDIT_BUREAU_MON':'count_features', 'AMT_REQ_CREDIT_BUREAU_QRT':'count_features', 'AMT_REQ_CREDIT_BUREAU_YEAR':'count_features',
                'bureau_bureau_cnt':'count_features', 'bureau_CREDIT_DAY_OVERDUE_sum':'amount_feautres', 'bureau_DAYS_CREDIT_ENDDATE_sum':'amount_feautres', 'bureau_DAYS_ENDDATE_FACT_sum':'amount_feautres', 'bureau_DAYS_CREDIT_sum':'amount_feautres', 'bureau_DAYS_CREDIT_UPDATE_sum':'amount_feautres', 'bureau_CNT_CREDIT_PROLONG_sum':'amount_feautres', 'bureau_AMT_CREDIT_SUM_sum':'amount_feautres', 'bureau_AMT_ANNUITY_sum':'amount_feautres', 'bureau_AMT_CREDIT_SUM_DEBT_sum':'amount_feautres', 'bureau_AMT_CREDIT_SUM_LIMIT_sum':'amount_feautres', 'bureau_AMT_CREDIT_MAX_OVERDUE_sum':'amount_feautres', 'bureau_MONTHS_BALANCE_max_sum':'amount_feautres', 'bureau_MONTHS_BALANCE_cnt_sum':'amount_feautres', 'bureau_STATUS_cntd_sum':'amount_feautres', 'bureau_AMT_CREDIT_MAX_OVERDUE_max':'amount_feautres', 'bureau_CNT_CREDIT_PROLONG_max':'amount_feautres', 'bureau_DAYS_CREDIT_max':'amount_feautres', 'bureau_MONTHS_BALANCE_max_max':'amount_feautres', 'bureau_STATUS_cntd_max':'amount_feautres', 'bureau_CREDIT_CURRENCY_cnt':'amount_feautres', 'bureau_CREDIT_ACTIVE_cnt':'amount_feautres', 'bureau_CREDIT_TYPE_cnt':'amount_feautres', 'bureau_STATUS_set_cnt':'amount_feautres', 'bureau_latest_STATUS_cnt':'amount_feautres', 'bureau_CREDIT_CURRENCY_set':'category_features', 'bureau_CREDIT_ACTIVE_set':'category_features', 'bureau_CREDIT_TYPE_set':'category_features', 'bureau_latest_STATUS_set':'category_features', 'bureau_STATUS_set':'category_features',
                'cash_MONTHS_BALANCE_min':'count_features', 'cash_MONTHS_BALANCE_max':'count_features', 'cash_MONTHS_BALANCE_cnt':'count_features', 'cash_CNT_INSTALMENT_min':'count_features', 'cash_CNT_INSTALMENT_max':'count_features', 'cash_CNT_INSTALMENT_set':'category_features', 'cash_CNT_INSTALMENT_FUTURE_min':'count_features', 'cash_CNT_INSTALMENT_FUTURE_max':'count_features', 'cash_NAME_CONTRACT_STATUS_cntd':'count_features', 'cash_NAME_CONTRACT_STATUS_set':'category_features', 'cash_NAME_CONTRACT_STATUS_latest':'category_features', 'cash_SK_DPD_max':'count_features', 'cash_SK_DPD_cnt0':'count_features', 'cash_SK_DPD_DEF_max':'count_features', 'cash_SK_DPD_DEF_cnt0':'count_features', 'cash_SK_DPD_diff_max':'count_features', 'cash_SK_DPD_diff_cnt0':'count_features',
               }

In [13]:
category_features = [f for f in feature_dict if feature_dict[f] == 'category_features' ]
boolean_features = [f for f in feature_dict if feature_dict[f] == 'boolean_features' ]
amount_feautres = [f for f in feature_dict if feature_dict[f] == 'amount_feautres' ]
normal_features = [f for f in feature_dict if feature_dict[f] == 'normal_features' ]
count_features = [f for f in feature_dict if feature_dict[f] == 'count_features' ]

In [14]:

minmax = MinMaxScaler().fit(pd.concat([train[amount_feautres+count_features],test[amount_feautres+count_features]],axis = 0).fillna(0))

train[amount_feautres+count_features] = pd.DataFrame(minmax.transform(train[amount_feautres+count_features].fillna(0)), columns = amount_feautres+count_features)
test[amount_feautres+count_features] = pd.DataFrame(minmax.transform(test[amount_feautres+count_features].fillna(0)), columns = amount_feautres+count_features)


In [15]:
# bin
bins = 5
for rate in normal_features+amount_feautres+count_features:
    qcut = pd.qcut(train[rate],bins,retbins=True,duplicates='drop')[1]
    if len(qcut) <=2:
        continue
    qcut[0] = 0
    qcut[-1] = 1
    train[rate +'_bin'] = [rate + '_' + str(np.where(qcut>= r)[0][0]) if not np.isnan(r) else 'miss' for r in train[rate]]
    test[rate +'_bin'] =  [rate + '_' +  str(np.where(qcut>= r)[0][0]) if not np.isnan(r) else 'miss' for r in test[rate]]
    category_features.append(rate +'_bin')


In [16]:
taget_encoder = ce.TargetEncoder(cols=category_features + boolean_features)
taget_encoder.fit(train[category_features + boolean_features], train.TARGET)

onehot_encoder = ce.OneHotEncoder(cols = category_features + boolean_features)
onehot_encoder.fit(train[category_features + boolean_features])

ordinal_encoder = ce.OrdinalEncoder(cols = category_features)
ordinal_encoder.fit(train[category_features])

X_train = pd.concat([
    train[normal_features+amount_feautres+count_features].fillna(0),
    taget_encoder.transform(train[category_features + boolean_features]),
    onehot_encoder.transform(train[category_features + boolean_features]),
    ordinal_encoder.transform(train[category_features]),
], axis = 1)
y_train = train.TARGET


X_test = pd.concat([
    test[normal_features+amount_feautres+count_features].fillna(0),
    taget_encoder.transform(test[category_features + boolean_features]),
    onehot_encoder.transform(test[category_features + boolean_features]),
    ordinal_encoder.transform(test[category_features]),
], axis = 1)

TargetEncoder(cols=['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START', 'ORGANIZATION_TYPE', ... 'FLAG_DOCUMENT_17', 'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21'],
       drop_invariant=False, handle_unknown='impute', impute_missing=True,
       min_samples_leaf=1, return_df=True, smoothing=1, verbose=0)

OneHotEncoder(cols=['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START', 'ORGANIZATION_TYPE', ... 'FLAG_DOCUMENT_17', 'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21'],
       drop_invariant=False, handle_unknown='impute', impute_missing=True,
       return_df=True, verbose=0)

OrdinalEncoder(cols=['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START', 'ORGANIZATION_TYPE', ...OCIAL_CIRCLE_bin', 'DAYS_LAST_PHONE_CHANGE_bin', 'AMT_REQ_CREDIT_BUREAU_YEAR_bin', 'bureau_cnt_bin'],
        drop_invariant=False, handle_unknown='impute', impute_missing=True,
        mapping=[{'col': 'NAME_CONTRACT_TYPE', 'mapping': [('Cash loans', 0), ('Revolving loans', 1)]}, {'col': 'CODE_GENDER', 'mapping': [('M', 0), ('F', 1), ('XNA', 2)]}, {'col': 'NAME_TYPE_SUITE', 'mapping': [('Unaccompanied', 0), ('Family', 1), ('Spouse, partner', 2), ('Children', 3), ('Other_A', 4), (n...au_cnt_3', 1), ('bureau_cnt_2', 2), ('bureau_cnt_0', 3), ('bureau_cnt_1', 4), ('bureau_cnt_5', 5)]}],
        return_df=True, verbose=0)

In [17]:
X_train.shape
X_train.head()

(307511, 1468)

,REGION_POPULATION_RELATIVE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,...,CREDIT_TYPE_cnt_bin,STATUS_set_cnt_bin,latest_STATUS_cnt_bin,CNT_CHILDREN_bin,CNT_FAM_MEMBERS_bin,OBS_30_CNT_SOCIAL_CIRCLE_bin,OBS_60_CNT_SOCIAL_CIRCLE_bin,DAYS_LAST_PHONE_CHANGE_bin,AMT_REQ_CREDIT_BUREAU_YEAR_bin,bureau_cnt_bin
0,0.018801,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,...,0,0,0,0,0,0,0,0,0,0
1,0.003541,0.311267,0.622246,0.000000,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,...,0,1,1,0,1,1,1,1,1,1
2,0.010032,0.000000,0.555912,0.729567,0.0000,0.0000,0.0000,0.0000,0.0000,0.00,...,1,1,1,0,0,2,2,1,1,2
3,0.008019,0.000000,0.650442,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.00,...,2,1,1,0,1,0,0,1,1,3
4,0.028663,0.000000,0.322738,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.00,...,1,1,1,0,0,2,2,0,1,4


In [23]:
X_train_cv, X_test_cv, y_train_cv, y_test_cv = train_test_split(X_train, y_train, test_size = 0.2, random_state = 42)

lr = LogisticRegression(C = 0.1, penalty = 'l1', random_state = 42)

# pipeline = Pipeline([
#     ('select', SelectKBest(chi2, k=300)),
#     ('clf', lr)])

lr.fit(X_train_cv, y_train_cv)
roc_auc_score(y_train_cv, lr.predict_proba(X_train_cv)[:,1]), roc_auc_score(y_test_cv, lr.predict_proba(X_test_cv)[:,1])

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

(0.75858179107330992, 0.7574009288775998)

In [24]:
lr = LogisticRegression(C = 0.1, penalty = 'l1', random_state = 42)
lr.fit(X_train, y_train)
roc_auc_score(y_train, lr.predict_proba(X_train)[:,1])

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

0.75961836005359584

In [25]:
sub = pd.concat([
    test.SK_ID_CURR,
    pd.DataFrame({'TARGET':lr.predict_proba(X_test)[:,1]})
], axis = 1)


In [26]:
sub.head()
sub.to_csv('./data/submission/third_20180526.csv', index = False)

,SK_ID_CURR,TARGET
0,100001,0.038323
1,100005,0.108996
2,100013,0.025333
3,100028,0.023132
4,100038,0.178688


In [27]:
pd.concat([X_train,pd.DataFrame(y_train, columns = ['TARGET'])], axis = 1).to_csv('./data/input/train.csv', index = False)
# y_train.to_csv('./data/input/y_train.csv', index = False)
X_test.to_csv('./data/input/test.csv', index = False)


In [35]:
entropy_select = mutual_info_classif(X_train, y_train)
mutual_info_selected = X_train.columns[entropy_select >= sorted(entropy_select,reverse = True)[99]]

f_var = X_train.apply(np.std,axis = 0)
var_selected = X_train.columns[f_var >= f_var.sort_values(ascending=False)[99]]


len(set(mutual_info_selected.tolist() + var_selected.tolist()))

In [52]:
ext_tree = ExtraTreesClassifier(n_estimators=500,random_state=42)

SelectFromModel(ext_tree)

179